In [ ]:
def get_other_lines( cas: Cas, SofaID: str , list_value_between_tagtype: list, root_paragraph ) -> list : 
    
    '''
    Convert cas with paragraph annotations to a nested lists to be used for annotation of reporting obligations.
    Structure of the nested lists is as follows:
    ->p   new_root (i.e. start of enumeration)
    ->p
    ->p
    [->p  new_root
    [->p  new_root
    ->p]]
    '''

    lines=[]
    position=0
    paragraphs_covering_tag=None
    new_root_paragraph=None
    
    for tag in list_value_between_tagtype:
        
        #check if tag is a "p", and if it was not part of one of the already nested p's.
        if not tag.tagName == 'p' or tag.begin < position:
            continue
            
        paragraphs_covering_tag= ['root'] + list(cas.get_view( SofaID ).select_covering( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph", tag ))
     
        #If no enumeration detected yet, set root_paragraph to 'root'.
        if root_paragraph=='root': 
            index_root=0
        else:
            index_root=get_index_typesystem_elements_in_list( paragraphs_covering_tag[1:], root_paragraph.xmiID )+1
        
        #sanity check
        assert index_root >= 0

        #Start of new sublist detected (i.e. the root_paragraph is no longer the "deepest" paragraph). Always go for the deepest root.
        if paragraphs_covering_tag[ index_root+1: ]:
            new_root_paragraph=paragraphs_covering_tag[ index_root+1 ]
            sublines, position=get_other_lines( cas, SofaID, cas.get_view( SofaID ).select_covered( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" \
                                                                     , new_root_paragraph ) , new_root_paragraph  )
            lines.append( sublines )
            
        else:
            lines.append(  tag.get_covered_text() )
            position=tag.end
            
    return lines, position

#helper function
def get_index_typesystem_elements_in_list( list_typesystem_elements: list, xmiID: int  ):
    
    '''
    Helper function to get index of certain typesystem element (with given xmiID) in list of typesystem elements
    Necessary because list_typesystem_elements.index( element ) does not return correct index,
    because all elements in list_typesystem_elements are considered equal (due to bug in dkpro-cassis)
    '''
    
    for index, item in enumerate(  list_typesystem_elements ):
        if item.xmiID == xmiID:
            return index
        
    return -1